In [19]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow import keras


In [20]:
# 1. Charger le modèle d'émotion pré-entraîné
model = keras.models.load_model("../models/CNN.keras")

c:\Users\germd\Desktop\Emoji-Face-Detection\.venv\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:

# 2. Initialiser MediaPipe pour la détection de visage
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [22]:
# 3. Dictionnaire émotion -> emoji (ajuster les chemins selon vos fichiers)
EMOJI_MAP = {
    0: ("colere", cv2.imread("../images/emojis/angry.png", cv2.IMREAD_UNCHANGED)),
    1: ("degout", cv2.imread("../images/emojis/disgust.png", cv2.IMREAD_UNCHANGED)),
    2: ("peur", cv2.imread("../images/emojis/fear.png", cv2.IMREAD_UNCHANGED)),
    3: ("heureux", cv2.imread("../images/emojis/happy.png", cv2.IMREAD_UNCHANGED)),
    4: ("neutre", cv2.imread("../images/emojis/neutral.png", cv2.IMREAD_UNCHANGED)),
    5: ("triste", cv2.imread("../images/emojis/sad.png", cv2.IMREAD_UNCHANGED)),
    6: ("surprise", cv2.imread("../images/emojis/surprised.png", cv2.IMREAD_UNCHANGED))
}

In [23]:
# Vérifiez que tous les emojis sont chargés
for emotion_id, (label, emoji) in EMOJI_MAP.items():
    if emoji is None:
        raise FileNotFoundError(f"Emoji {label} (ID: {emotion_id}) non trouvé !")

In [24]:
# 4. Préprocesser l'image pour le modèle CNN
def preprocess_face(face_img, target_size=(48, 48)):
    # Convertir en niveaux de gris si le modèle le requiert
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    face_img = cv2.resize(face_img, target_size)
    face_img = face_img.astype("float32") / 255.0
    face_img = np.expand_dims(face_img, axis=-1)  # Ajouter une dimension de canal
    face_img = np.expand_dims(face_img, axis=0)   # Ajouter une dimension de batch
    return face_img

In [25]:
# 5. Fonction pour superposer l'emoji avec transparence
def overlay_emoji(frame, emoji, x, y, w, h):
    if emoji is None or emoji.size == 0 or w <=0 or h <=0:
        return frame
    
    # Redimensionner l'emoji EXACTEMENT à la taille du visage
    try:
        emoji = cv2.resize(emoji, (w, h))
        alpha = emoji[:, :, 3] / 255.0
        for c in range(3):
            # Extraire la ROI de la frame avec les mêmes dimensions que l'emoji
            frame_roi = frame[y:y+h, x:x+w, c]
            if frame_roi.shape != alpha.shape:
                # Ajuster alpha aux dimensions de frame_roi si nécessaire
                alpha = cv2.resize(alpha, (frame_roi.shape[1], frame_roi.shape[0]))
            frame[y:y+h, x:x+w, c] = alpha * emoji[:, :, c] + (1 - alpha) * frame_roi
    except Exception as e:
        print(f"Erreur d'overlay : {e}")
    return frame

In [27]:
# 6. Capture vidéo en direct
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Détection de visage avec MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            # Récupérer les coordonnées du bounding box
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x = max(0, int(bbox.xmin * w))
            y = max(0, int(bbox.ymin * h))
            face_w = max(10, int(bbox.width * w))  # Éviter des valeurs <=0
            face_h = max(10, int(bbox.height * h))

            # Ajuster la largeur/hauteur si le visage dépasse de l'écran
            face_w = min(face_w, w - x)
            face_h = min(face_h, h - y)

            # Vérifier que la région est valide
            if face_w <= 0 or face_h <= 0:
                continue

            # Découper et préprocesser le visage
            face_roi = frame[y:y+face_h, x:x+face_w]

            # Si la ROI est vide (erreur MediaPipe), ignorer
            if face_roi.size == 0 or face_roi.shape[0] != face_h or face_roi.shape[1] != face_w:
                continue
            
            processed_face = preprocess_face(face_roi)

            # Prédiction d'émotion
            predictions = model.predict(processed_face)
            emotion_id = np.argmax(predictions)
            emotion_label, emoji = EMOJI_MAP[emotion_id]

            # Superposer l'emoji
            frame = overlay_emoji(frame, emoji, x, y, face_w, face_h)

    # Afficher le résultat
    cv2.imshow('Emoji Emotion Overlay', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━